# Lista de Exercício 6
### Introdução ao Processamento Digital de Imagens (SEL0449/SEL5895)

**Instruções:**

 1. Esta lista consiste em 5 exercícios. O peso considerado para a nota final da lista está descrito ao lado do enunciado.
 1. Deve-se colocar comentários nos códigos desenvolvidos.
 1. As perguntas devem ser respondidas também como comentários no arquivo.
 1. Colocar seu nome e número USP abaixo.
 1. Quaisquer problemas na execução das listas, entrar em contato com os monitores.
 1. Não retirar os enunciados da lista.
 1. Depois de terminados os exercícios, deve ser gerado um arquivo **extensão .ipynb** para ser enviado ao professor pelo E-DISCIPLINAS da disciplina até a data máxima de entrega.
 1. Caso não seja enviado, o aluno ficará sem nota.


---



 <table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/LAVI-USP/SEL0449-SEL5895_2024/blob/main/praticas/Lista_de_Exercicio_6.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Executar no Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/LAVI-USP/SEL0449-SEL5895_2024/blob/main/praticas/Lista_de_Exercicio_6.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Ver codigo fonte no GitHub</a>
  </td>
</table>


`Nome: `

`Número USP: `

### Introdução:

Nesta lista de exercícios vamos estudar sobre filtros aplicados no domínio da frequência. Primeiramente vamos importar as bibliotecas que iremos utilizar:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from mpl_toolkits import mplot3d

#### **Atenção**: os códigos abaixo são para fazer o download das imagens necessárias para a prática. EXECUTE-OS!

In [2]:
import urllib.request

try:
  urllib.request.urlretrieve("https://github.com/LAVI-USP/SEL0449-SEL5895_2024/blob/main/imagens/pratica_06/towerbridge.tif?raw=true", "towerbridge.tif")
except:
  print("[ERRO] Não foi possível fazer o download das imagens dessa prática. Entre em contato com o monitor")

try:
  urllib.request.urlretrieve("https://github.com/LAVI-USP/SEL0449-SEL5895_2024/blob/main/imagens/pratica_06/arvore_mod.tif?raw=true", "arvore_mod.tif")
except:
  print("[ERRO] Não foi possível fazer o download das imagens dessa prática. Entre em contato com o monitor")

try:
  urllib.request.urlretrieve("https://github.com/LAVI-USP/SEL0449-SEL5895_2024/raw/main/imagens/pratica_06/hill.tif", "hill.tif")
except:
  print("[ERRO] Não foi possível fazer o download das imagens dessa prática. Entre em contato com o monitor")

try:
  urllib.request.urlretrieve("https://github.com/LAVI-USP/SEL0449-SEL5895_2024/raw/main/imagens/pratica_06/hiIzt.png", "car.png")
except:
  print("[ERRO] Não foi possível fazer o download das imagens dessa prática. Entre em contato com o monitor")

try:
  urllib.request.urlretrieve("https://github.com/LAVI-USP/SEL0449-SEL5895_2024/raw/main/imagens/pratica_06/parede.tif", "parede.tif")
except:
  print("[ERRO] Não foi possível fazer o download das imagens dessa prática. Entre em contato com o monitor")

### PARTE I - Filtragem no domínio da frequência

A filtragem no domínio da frequência consiste em modificar a transformada de Fourier de uma imagem e depois calcular a transformada inversa para obter o resultado processado.

O grau de dificuldade na construção dos filtros diminui quando se trabalha no domínio da frequência, entretanto, a proximidade dos períodos na convolução pode causar o erro de wraparound (efeito de borda). Para solucionar esse problema, é necessário fazer um padding na imagem - lembrando que algoritmos DFT tendem a executar mais rapidamente arranjos de tamanho par, ou seja, normalmente se utiliza o dobro das dimensões M e N da imagem. As Figuras 1 e 2 ilustram os efeitos provocados, no borramento da imagem, tanto devido a não utilização de padding quanto da aplicação zero padding.

Lembre-se que, afim de preservar informações sobre as bordas, normalmente utiliza-se do padding simétrico, e não o padding com zeros.


<center><img src="https://github.com/LAVI-USP/SEL0449-SEL5895_2024/blob/main/imagens/pratica_06/wraparound.PNG?raw=true" style="width:836px;height:266px;"></center>

<center><caption><b> Figura 1:</b> Periodicidade inerente às imagens 2D na utilização da DFT.</b></caption></center>
<caption><center> Referência: Gonzalez and Woods, Digital Image Processing 3rd.</center></caption></br>


<center><img src="https://github.com/LAVI-USP/SEL0449-SEL5895_2024/blob/main/imagens/pratica_06/wraparound_0.PNG?raw=true" style="width:836px;height:266px;"></center>

<center><caption><b> Figura 2:</b> Em b, resultado do borramento de a sem zero padding. Em c, resultado do borramento de a com zero padding. </b></caption></center>
<caption><center> Referência: Gonzalez and Woods, Digital Image Processing 3rd.</center></caption></br>

A função [np.fft.ff2](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft2.html) - utilizada na lista anterior - oferece, por meio do parâmetro s, a opção de preenchimento (padding), sendo (Mf,Nf) normalmente igual a (2M, 2N) tal que M e N representam as dimensões da imagem. Para mais detalhes, consulte a documentação.

```python
f = np.fft.fft2(img,s=(Mf,Nf))
fshift = np.fft.fftshift(f)
magnitude_spectrum = 20*np.log(np.abs(fshift)+1.)
```

Atenção! Note as novas dimensões do espectro de frequência. Após a transformada inversa, será necessário recortar a imagem, resultando na imagem com as dimensões originais M e N, descartando as informações adjacentes.

### 1) Filtros PASSA-BAIXA E PASSA-ALTA no domínio da frequência (Nota: 2.5/10.0)

Bordas e outras transições abruptas de intensidade (como o ruído) em uma imagem contribuem significativamente para o conteúdo de alta frequência de sua transformada de Fourier. Dessa forma, a suavização (borramento) é obtida no domínio da frequência pela atenuação das altas frequências - filtros passa-baixa.

Os filtros podem ser projetados de forma a apresentar transições mais abruptas ou mais suavizadas. Consideraremos apenas os filtros radialmente simétricos - deslocamento de fase zero - sendo D<sub>0</sub> o raio a partir da origem e n a ordem do filtro:
<p></br>

Filtro passa-baixa IDEAL:

$$ H_{LPideal}(u,v) = 1,~se~D(u,v)~\le~D0$$<br/>
$$ H_{LPideal}(u,v) = 0,~se~D(u,v)~\gt~D0$$<br/>

Filtro passa-baixa BUTTERWORTH:

$$ H_{LPButterworth}(u,v) = \frac{1}{{1+\big[\frac{D(u,v)}{D_0}}\big]^{2n}}$$<br/>

Filtro passa-baixa GAUSSIANO:

$$ H_{LPGaussiano}(u,v) = e^{\frac{-D^2(u,v)}{2D_0^2}}$$<br/>

Já o aguçamento das imagens pode ser obtido pela filtragem passa-alta, atenuando componentes de baixa frequência sem afetar as informações de alta frequência na transformada de Fourier. Os filtros passa-alta podem ser obtidos a partir dos respectivos passa-baixa, considerando a equação:

$$ H_{HP}(u,v) = 1 - H_{LP}(u,v)$$<br/>


**DICA:** para visualizar os filtros em plots 3D, você pode utilizar o toolkit já incluso na instalação da biblioteca matplotlib (incluso na importação inicial deste notebook). Para isto, é necessário manipular os dados do filtro, de forma que seja construída uma rede X-Y (contendo as coordenadas) em que serão plotadas as intensidades (Z). Considere um filtro de dimensões Mf e Nf e intensidades I. Para criar a rede, podemos utilizar o comando [np.linspace](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) tendo o 0 (zero) como valor inicial, Mf como valor final e com número de amostras equivalente a Mf. Da mesma forma com Nf.

A seguir, um exemplo de subplot utilizando a função plot_wireframe e plot_surface. Consulte o [link](https://matplotlib.org/stable/tutorials/toolkits/mplot3d.html) para mais detalhes.

```python
x = np.linspace(0, Mf, Mf)
y = np.linspace(0, Nf, Nf)

X, Y = np.meshgrid(x, y)
Z = I

fig = plt.figure(figsize = (20,10))
fig.suptitle('Filtros 3D')

ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.plot_wireframe(X,Y,Z)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');
ax.set_title('wireframe plot');

ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot_surface(X, Y, Z, rstride=2, cstride=2,cmap='viridis', edgecolor='none')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');
ax.set_title('surface plot');

plt.show()
```


**Exercício.** Considerando que a imagem 'towerbridge.tif' foi digitalizada com **resolução de 300DPI**, realize as seguintes tarefas:

**a.** Visualize a imagem original e o espectro de Fourier (domínio da frequência). **Lembre-se do padding.**

In [ ]:
## -- Seu código começa AQUI -- ##


## -- Seu código termina AQUI -- ##


**b.** Crie um filtro **passa-baixa circular do tipo "Ideal"** com frequência de corte igual a 1,7 ciclos/mm. Aplicar à imagem e comentar o resultado. Mostre a imagem original, a magnitude do filtro criado, a imagem filtrada e o espectro de frequências filtrado.

In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

**c.** Crie um filtro **passa-baixa circular do tipo "Butterworth"** com n=2 e frequência de corte igual a 1,7 ciclos/mm. Aplicar à imagem e comentar o resultado.  Mostre a imagem original, a magnitude do filtro criado, a imagem filtrada e o espectro de frequências filtrado.

In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

**d.** Crie um filtro **passa-baixa circular do tipo "Gaussiano"** com frequência de corte igual a 1,7 ciclos/mm. Aplicar à imagem e comentar o resultado.  Mostre a imagem original, a magnitude do filtro criado, a imagem filtrada e o espectro de frequências filtrado.

In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

**e.** Crie os **filtros passa-alta** com as mesmas frequências de corte e características especificadas em b, c e d. Em cada um, mostre a imagem original, a magnitude do filtro criado, a imagem filtrada e o espectro de frequências filtrado.

In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

In [ ]:
# COMENTÁRIOS:
#

**f.** Visualize os filtros PASSA-BAIXA e PASSA-ALTA criados (nos itens acima) em plots 3D (Wireframe e Surface).

In [ ]:
## FILTROS IDEAIS
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

In [ ]:
## FILTROS BUTTERWORTH
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

In [ ]:
## FILTROS GAUSSIANO
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

### 2) Filtros PASSA-BANDA E REJEITA-BANDA no domínio da frequência (Nota: 2.5/10.0)

Existem algumas aplicações nas quais é interessante processar bandas específicas de frequências. Para tanto, utiliza-se filtros conhecidos como filtros rejeita-banda e passa-banda. Assim como mostrado anteriormente, a transição pode ser feita de modo abrupto ou suavizado. As equações a seguir referem-se a filtros rejeita-banda (BR de *bandreject*):

Filtro rejeita-baixa IDEAL:

$$ H_{BRideal}(u,v) = 0, \;se\;D_0 - \frac{W}{2} \le D \le D_0 + \frac{W}{2}$$<br/>
$$ H_{BRideal}(u,v) = 1,\; para\;todos\;outros\;casos$$<br/>

Filtro rejeita-baixa BUTTERWORTH:

$$ H_{BRButterworth}(u,v) = \frac{1}{1+\big[\frac{D \cdot W}{D^2-D_0^2}\big]^{2n}}$$<br/>

Filtro rejeita-baixa GAUSSIANO:

$$ H_{BRGaussiano}(u,v) = 1-e^{-\big(\frac{D^2-D_0^2}{D \cdot W}\big)^2}$$<br/>


Um filtro passa-banda (BP de *bandpass*) é obtido a partir do respectivo rejeita-banda, conforme equação abaixo:

$$ H_{BP}(u,v) = 1 - H_{BR}(u,v)$$<br/>



**Exercício.** Considerando que a imagem 'arvore_mod.tif' foi digitalizada com **resolução de 400DPI**, realize as seguintes tarefas:

**a.** Visualize a imagem original e o espectro de Fourier (domínio da frequência). **Lembre-se do padding.**

In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

**b.** Encontre o centro do espectro de frequências. Plote um recorte central. Encontre o tamanho de recorte que permita melhor visualização dos pontos mais evidentes simétricos em relação ao centro.

In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##


**c.** Crie um **filtro rejeita-banda circular do tipo "ideal"** para ser aplicado à imagem. Os parâmetros de frequência de corte em ciclos/mm e a largura de banda em ciclos/mm devem ser escolhidos preservando ao máximo o conteúdo da imagem. Utilize a referência do exercício "b" para auxiliar na escolha dos parâmetros e comente justificando.

Crie um **filtro passa-banda circular do tipo "ideal"** com os mesmos parâmetros de frequência de corte para visualizar as componentes removidas.

Comente o resultado obtido.

In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

# Comentários:

**d.** Visualize os filtros PASSA-BANDA e REJEITA-BANDA criados em plots 3D (Wireframe e Surface).

In [ ]:
## FILTROS IDEAIS
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

### PARTE II - Outros filtros no domínio da frequência

Vamos, agora, estudar sobre alguns filtros que podem ser aplicados no domínio da frequência visando melhorar a qualidade das imagens. Lembre-se que o sistemas de coordenadas no domínio da frequência é um pouco diferente:

<center><img src="https://github.com/LAVI-USP/SEL0449-SEL5895_2024/raw/main/imagens/pratica_06/ExemploCoordenadas.png" ></center>

<center><caption><b>Figura 1:</b> Sistemas de coordenadas no domínio original da imagem (esquerda) e domínio de Fourie (direita).</b></caption></center>

### 3) Filtro notch reject (Nota: 1.5/10.0):

Em qual situação devo utilizar um filtro notch? Filtros notch são frequentemente utilizados para rastrear e retirar componentes senoidais de sinais. Nas imagens isto não é diferente.

Filtragem notch reject é usada para eliminar efeitos de ruídos periódicos. Estes filtros são capazes de rejeitar regiões específicas em torno de uma
frequência pré-definida, assim, sua utilização é recomendada quando o sinal a ser atenuado é bem definido.

<center><img src="https://github.com/LAVI-USP/SEL0449-SEL5895_2024/raw/main/imagens/pratica_06/ExemploFiltroNotch2.png" width="380" height="380"></center>

<center><caption><b> Figura 2:</b> Exemplo de filtragem com Notch em ruído periódico.</b></caption></center>
<caption><center> </center></caption>

**Exercício:**

Nesta etapa, vamos trabalhar com a imagem ```car.png```. Considere que a imagem foi digitalizada com resolução de 400 DPI.

1. Carregue o arquivo de imagem ```car.png``` e mostre-a.
2. Passe esta imagem para o domínio da frequência (Espectro de Fourier) utilizando as funções ```np.fft.fft2``` e ```np.fft.fftshift``` (**Lembre-se do padding.**). Em seguida, mostre a magnitude do espectro.
3. Para visualizar e compreender melhor, vamos dar um "zoom" na imagem no domínio da frequência. Selecione a região ```[0:800,200:360]``` e mostre o resultado desse corte. Comente o que são os "pontos brancos mais evidentes" no espectro.
4.  Faça um filtro do tipo notch reject butterworth de ordem ```ord = 4``` com raio ```D0 = 9``` para remover o ruído da seguinte forma:
 *   4 pontos mais próximos do centro do espectro
     *   Encontre o centro dos "pontos brancos" mais próximo ao centro do espectro (pode ser de forma empírica/visual)
     *   Crie o filtro com base no centro desses 4 pontos encontrados
 *   8 pontos mais próximos do centro do espectro
     *   Encontre o centro dos "pontos brancos" mais próximo ao centro do espectro (pode ser de forma empírica/visual)
     *   Crie o filtro com base no centro desses 8 pontos encontrados

 Os centros do filtro devem coincidir com o centro dos pontos encontrados no
espectro da frequência.

5. Para cada situação, moste a magnitude do filtro criado. Após aplicação do filtro construído, mostre, também, a magnitude do espectro de Fourier resultante, bem como a imagem filtrada, agora, em seu domínio original.
6. Comente os resultados


In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

## Comentários:

### 4) Realce (Nota: 1.5/10.0):

Nesta parte da lista iremos trabalhar com o filtro de realce. Para isso, utilizaremos a imagem ```parede.tif```. Considere que a imagem foi digitalizada com resolução de 500 DPI.
Dica: Como este filtro realça principamente os detalhes, é interessante analisar as imagens sempre com ZOOM!

1. Carregue a imagem “parede.tif” e mostre-a.
2. Visualizar a imagem no domínio da frequência (Espectro de Fourier). **Lembre-se do padding.**
4. Criar um filtro de realce (30%), utilizando passa-alta do tipo “Butterworth” (n = 2), com frequência de corte 3,0 ciclos/mm. Aplicar na imagem e verificar o resultado.
5. Criar um filtro do tipo "Gaussiano", no domínio da frequência, que realce em 40% todas as frequências entre 2,0 e 4,0 ciclos/mm. Aplicar na imagem e verificar o resultado no domínio do espaço.

In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

### 5) Filtragem Homomórfica (Nota: 2.0/10.0)

Agora vamos trabalhar o realce corrigindo uma imagem com iluminação não uniforme. Para isso, vamos trabalhar com a figura "***hill.tif***". Considere que essa imagem foi digitalizada com resolução de 300 DPI.

1. Carregue o arquivo de imagem ```hill.tif``` e mostre-a.
2. Aplique uma correção Logarítmica à imagem original a fim de fazer uma correção de contraste. *Dica: após essa correção, converta a imagem_Log para float32*.
3. Construa padding simétrico (espelhamento) para eliminar o efeito de *wrap around*. Para fazer isso, utilize a função ```np.pad```.
4. Agora, vamos passar a imagem com *padding* para o domínio da frequência e visualizar o espectro de frequências.
5. Em seguida, crie um filtro do tipo homomórfico, com ```γL = 0,5``` e ```γH = 1,5```, com variação Gaussiana e com frequência de corte de 2,5 ciclos/mm e aplique na imagem. Mostre o filtro construido.
6. Aplique o filtro homomórfico criado sobre a imagem no domínio da frequência.
7. Aplique a transformada inversa para retornar a imagem ao domínio original.
8. Aplique uma correção exponencial (```np.exp```) para corrigir a transformação logarítmica no início. Por fim, faça um alargamento de contraste sobre a imagem filtrada no domínio original e mostre-a.
9. Comentar os resultados encontrados após aplicação dos filtros.


In [ ]:
## -- Seu código começa AQUI -- ##

## -- Seu código termina AQUI -- ##

## Comentários: